# Wiki-Split

In [1]:
from datasets import load_dataset

dataset = load_dataset("wiki_split")

Using custom data configuration default
Reusing dataset wiki_split (/home/bhadresh/.cache/huggingface/datasets/wiki_split/default/0.1.0/83151e5f3f5a622601303c2f562ffda5502643e38f3f1fee48999d0030f82b07)


RuntimeError: KeyboardInterrupt: 

In [ ]:
dataset

## Train data:

In [ ]:
train_dataset = dataset['train']
train_dataset

In [ ]:
# train_dataset = train_dataset.map(lambda x: {"complex_sentence": "sentence split: "+x["complex_sentence"]})
# train_dataset

In [ ]:
train_dataset = train_dataset.map(
    lambda x: {"target_sentences": x["simple_sentence_1"]+ " <sep> " +x["simple_sentence_2"]}, 
    remove_columns=['simple_sentence_1', 'simple_sentence_2'])

In [ ]:
train_dataset[0]

In [ ]:
train_dataset.to_csv('train_file.csv', index=False)

## Validation data:

In [ ]:
validation_dataset = dataset['validation']
validation_dataset

In [ ]:
validation_dataset = validation_dataset.map(
    lambda x: {"target_sentences": x["simple_sentence_1"]+ " <sep> " +x["simple_sentence_2"]}, 
    remove_columns=['simple_sentence_1', 'simple_sentence_2'])

In [ ]:
validation_dataset.to_csv('validation_file.csv', index=False)

## Predict Data

In [ ]:
test_dataset = dataset['test']
test_dataset = test_dataset.map(
    lambda x: {"target_sentences": x["simple_sentence_1"]+ " <sep> " +x["simple_sentence_2"]}, 
    remove_columns=['simple_sentence_1', 'simple_sentence_2'])
test_dataset.to_csv('test_file.csv', index=False)

In [ ]:
test_dataset

# Web Split

In [23]:
import pandas as pd

## test set

In [20]:
test_simple = pd.read_csv('web_split_seq2seq/test_simple.txt', sep='tab', header=None)
test_complex = pd.read_csv('web_split_seq2seq/test_complex.txt', sep='tab', header=None)
test_df = pd.concat([test_complex, test_simple], axis=1)
test_df.columns = ['complex_sentence', 'simple_sentence']
test_df.iloc[0]

complex_sentence    The American , Duncan Rouleau created the char...
simple_sentence     Baymax was created by Duncan Rouleau . Duncan ...
Name: 0, dtype: object

In [26]:
def modify_eval_data(df):
    simp_sen = []
    comp_sens = []
    simp_sens = []
    pre_comp_sen = df.iloc[0]['complex_sentence']
    for i in range(len(df)):
        if df.iloc[i]['complex_sentence']==pre_comp_sen:
            simp_sen.append(df.iloc[i]['simple_sentence'])
        else:
            simp_sens.append(simp_sen)
            comp_sens.append(pre_comp_sen)
        pre_comp_sen = df.iloc[i]['complex_sentence']
    print(len(comp_sens), len(comp_sens))
    new_df = pd.DataFrame()
    new_df['complex_sentence'] = comp_sens
    new_df['simple_sentence'] = simp_sens
    return new_df
test_dataset = modify_eval_data(test_df)
test_dataset.head()

929 929


,complex_sentence,simple_sentence
0,"The American , Duncan Rouleau created the char...",[Baymax was created by Duncan Rouleau . Duncan...
1,The musician Andrew White 's genre is alternat...,[Baymax was created by Duncan Rouleau . Duncan...
2,Above the Veil is from Australia and was prece...,[Baymax was created by Duncan Rouleau . Duncan...
3,10 Hygiea has an apoapsis of 523951582.33968 k...,[Baymax was created by Duncan Rouleau . Duncan...
4,Addiction journal is about addiction and is pu...,[Baymax was created by Duncan Rouleau . Duncan...


In [27]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   complex_sentence  929 non-null    object
 1   simple_sentence   929 non-null    object
dtypes: object(2)
memory usage: 14.6+ KB


In [32]:
test_dataset.to_json('web_split_seq2seq/test_file.json', orient='index')

tcmalloc: large alloc 4294967296 bytes == 0x2d33ec000 @  0x7f062eb22680 0x7f062eb42bdd 0x7f05790793e3 0x7f057907a648 0x7f057907a368 0x7f057907a285 0x7f057907a285 0x7f057907a7f1 0x7f0579077eaa 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x68cdc7 0x5ff5d4 0x5c3cb0 0x56aadf 0x501148 0x56c422 0x501148 0x56c422 0x501148 0x504d56
tcmalloc: large alloc 8589934592 bytes == 0x413bec000 @  0x7f062eb22680 0x7f062eb42bdd 0x7f05790793e3 0x7f057907a648 0x7f057907a368 0x7f057907a285 0x7f057907a285 0x7f057907a7f1 0x7f0579077eaa 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x68cdc7 0x5ff5d4 0x5c3cb0 0x56aadf 0x501148 0x56c422 0x501148 0x56c422 0x501148 0x504d56
tcmalloc: large alloc 7566508032 bytes == 0x1340a000 @  0x7f062eb22680 0x7f062eb43824 0x4f0f6a 0x7f0579077fa8 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x

In [35]:
test_dataset.to_csv('web_split_seq2seq/test_file.csv', index=False)

## Validation set

In [29]:
validation_simple = pd.read_csv('web_split_seq2seq/validation_simple.txt', sep='tab', header=None)
validation_complex = pd.read_csv('web_split_seq2seq/validation_complex.txt', sep='tab', header=None)
validation_df = pd.concat([validation_complex, validation_simple], axis=1)
validation_df.columns = ['complex_sentence', 'simple_sentence']
validation_df.iloc[0]

/home/bhadresh/preprocessor/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


complex_sentence    Alcatraz Versus the Scrivener 's Bones , writt...
simple_sentence     The book Alcatraz Versus the Evil Librarians i...
Name: 0, dtype: object

In [30]:
validation_dataset = modify_eval_data(validation_df)
validation_dataset.info()

953 953


,complex_sentence,simple_sentence
0,"Alcatraz Versus the Scrivener 's Bones , writt...",[The book Alcatraz Versus the Evil Librarians ...
1,"Alderney Airport serves the city of Alderney ,...",[The book Alcatraz Versus the Evil Librarians ...
2,The Icebreaker Aleksey Chrikov was built at th...,[The book Alcatraz Versus the Evil Librarians ...
3,The Alderney Airport serves the island of Alde...,[The book Alcatraz Versus the Evil Librarians ...
4,Adisham Hall is located in Sri Lanka and it 's...,[The book Alcatraz Versus the Evil Librarians ...


In [33]:
validation_dataset.to_json('web_split_seq2seq/validation_file.json', orient='index')

tcmalloc: large alloc 4294967296 bytes == 0x2d33ec000 @  0x7f062eb22680 0x7f062eb42bdd 0x7f05790793e3 0x7f057907a648 0x7f057907a368 0x7f057907a285 0x7f057907a285 0x7f057907a7f1 0x7f0579077eaa 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x68cdc7 0x5ff5d4 0x5c3cb0 0x56aadf 0x501148 0x56c422 0x501148 0x56c422 0x501148 0x504d56
tcmalloc: large alloc 8589934592 bytes == 0x413bec000 @  0x7f062eb22680 0x7f062eb42bdd 0x7f05790793e3 0x7f057907a648 0x7f057907a368 0x7f057907a285 0x7f057907a285 0x7f057907a7f1 0x7f0579077eaa 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x68cdc7 0x5ff5d4 0x5c3cb0 0x56aadf 0x501148 0x56c422 0x501148 0x56c422 0x501148 0x504d56
tcmalloc: large alloc 6968238080 bytes == 0x1340a000 @  0x7f062eb22680 0x7f062eb43824 0x4f0f6a 0x7f0579077fa8 0x5f2cc9 0x5f30ff 0x5705f6 0x5f5956 0x56acb6 0x568d9a 0x5f5b33 0x56bc9b 0x568d9a 0x50b868 0x56bc9b 0x568d9a 0x

In [34]:
validation_dataset.to_csv('web_split_seq2seq/validation_file.csv', index=False)

## Trainset

In [1]:
from dask import dataframe as dd

In [11]:
train_simple = dd.read_csv('web_split_seq2seq/train_simple.txt', sep='\n', header=None)
train_complex = dd.read_csv('web_split_seq2seq/train_complex.txt', sep='\n', header=None)

In [14]:
train_simple.head()

,0
0,The kidney bean is an ingredient in Bandeja pa...
1,Kidney beans are an ingredient in Bandeja pais...
2,Kidney beans are an ingredient in Bandeja pais...
3,The kidney bean is an ingredient in Bandeja pa...
4,The kidney bean is an ingredient in Bandeja pa...


In [15]:
train_complex.head()

,0
0,Bandeja pasta -LRB- Colombia -RRB- includes th...
1,Bandeja pasta -LRB- Colombia -RRB- includes th...
2,Bandeja pasta -LRB- Colombia -RRB- includes th...
3,Bandeja pasta -LRB- Colombia -RRB- includes th...
4,Bandeja pasta -LRB- Colombia -RRB- includes th...


In [24]:
train_df = pd.DataFrame()
train_df['complex_sentence'] = list(train_complex[0])
train_df['simple_sentence'] = list(train_simple[0])
train_df.to_csv('web_split_seq2seq/train_file.csv', index=False)
train_df.head()

,complex_sentence,simple_sentence
0,Bandeja pasta -LRB- Colombia -RRB- includes th...,The kidney bean is an ingredient in Bandeja pa...
1,Bandeja pasta -LRB- Colombia -RRB- includes th...,Kidney beans are an ingredient in Bandeja pais...
2,Bandeja pasta -LRB- Colombia -RRB- includes th...,Kidney beans are an ingredient in Bandeja pais...
3,Bandeja pasta -LRB- Colombia -RRB- includes th...,The kidney bean is an ingredient in Bandeja pa...
4,Bandeja pasta -LRB- Colombia -RRB- includes th...,The kidney bean is an ingredient in Bandeja pa...
